In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
display(application_df.head())

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value (500) and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Choose a cutoff value (1000) and create a list of classifications to be replaced
classifications_to_replace = list(class_value_counts [class_value_counts <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
display(dummies_df.head())

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
# Used 'stratify=y' to evenly distribute the trget variable (1's and 0's) amongst the traning and test data set. this gives us a more reflective dataset.
# Used 'test_size=.2' to give my training data set 80% ofthe data to work with.
# This will give my train set more data to train with to make it better at predicting.
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y,test_size=.2)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimisation 1 - Adding More Neurons

Accuracy = 73% (same)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Will be using all the input features (43).
units_1 = 80
units_2 = 80
input_features = len(X_train_scaled[0])
nn_model1 = tf.keras.models.Sequential()

# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Output layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10,081
Trainable params: 10,081
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model (will use epochs = 200)
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
858/858 [==============================] - 4s 2ms/step - loss: 0.5704 - accuracy: 0.7187
Epoch 2/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5576 - accuracy: 0.7271
Epoch 3/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7268
Epoch 4/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7260
Epoch 5/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5515 - accuracy: 0.7279
Epoch 6/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5497 - accuracy: 0.7298
Epoch 7/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7292
Epoch 8/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7299
Epoch 9/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7312
Epoch 10/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5475 - accura

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5785 - accuracy: 0.7373 - 692ms/epoch - 3ms/step
Loss: 0.5784764885902405, Accuracy: 0.7373178005218506


Optimisation 2 - Adding more layers

Accuracy = 73% (same)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Will be using all the input features (43).
units_1 = 80
units_2 = 30
units_3 = 20
input_features = len(X_train_scaled[0])
nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_3, activation = "relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 20)                620       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 6,591
Trainable params: 6,591
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5686 - accuracy: 0.7194
Epoch 2/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7277
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7282
Epoch 4/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7278
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7309
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7309
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7307
Epoch 8/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7309
Epoch 9/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7324
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5468 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5543 - accuracy: 0.7367 - 1s/epoch - 7ms/step
Loss: 0.5543180704116821, Accuracy: 0.7367346882820129


Optimisation 3 - Using Different Activation Functions (tanh) Accuracy = 73% (same)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Will be using all the input features (43).
units_1 = 80
units_2 = 30
input_features = len(X_train_scaled[0])
nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_2, activation = "tanh"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 3s 2ms/step - loss: 0.5698 - accuracy: 0.7199
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7268
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7267
Epoch 4/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7285
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7272
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7296
Epoch 7/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7307
Epoch 8/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7297
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7303
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5472 - accura

In [21]:
nn_model3.save('AlphabetSoupCharity_Optimization.h5')